In [1]:
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline
import transformers
import torch
import os
import evaluate

In [25]:
def image_feeder(folder_path):
    images = []
    files = []
    for file in os.listdir(folder_path):
        try:
            # new_name = f"cartoon_dog_and_cat_{file}"
            img_path = os.path.join(folder_path, file)
            # new_name_path = os.path.join(folder_path, new_name)

            # os.rename(img_path, new_name_path)

            img = Image.open(img_path)
            images.append(img)
            files.append(file)
        except IOError:
            print(f"Could not read image: {file}")
    return images, files

In [3]:
# Prepare upscaler pipeline
upscaler = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(upscaler, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
def upscaler(image, pipeline, prompt):
    low_res_img = image.resize((128, 128))
    
    upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]
    return upscaled_image

In [24]:
cartoon_cat_path = "../assets/Evaluation Dataset/Cartoon Dog and Cat"

images, files = image_feeder(cartoon_cat_path)

files

['3.jpg',
 '5.jpg',
 '4.png',
 '9.jpg',
 '6.jpg',
 '7.jpg',
 '10.png',
 '2.jpg',
 '8.jpg',
 '1.jpg']

In [2]:
import pandas as pd

def collect_files_and_labels(base_path):
    data = {'file_path': [], 'label': []}

    categories = {
        'Real Cat': 'cat',
        'Real Dog': 'dog',
        'Cartoon Cat': 'cat',
        'Cartoon Dog': 'dog'
    }

    for folder, label in categories.items():
        folder_path = os.path.join(base_path, folder)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                data['file_path'].append(file_path)
                data['label'].append(label)

    df = pd.DataFrame(data)
    return df

base_path = '/workspaces/ModelHubTest/experiments/assets/Evaluation Dataset'
df = collect_files_and_labels(base_path)

df.to_csv("../assets/Evaluation Dataset/evaluation_dataset.csv")

In [45]:
# prepare classification model

cat_dog_models = pd.read_pickle("/workspaces/ModelHubTest/experiments/cats_vs_dogs.pkl")
cat_dog_models.iloc[0]

model         danieltur/my_awesome_catdog_model
accuracy                                    1.0
dataset                          [cats_vs_dogs]
base_model    google/vit-base-patch16-224-in21k
likes                                         0
downloads                                    12
Name: 6, dtype: object

In [22]:
for model in cat_dog_models["model"]:
    print(model)

danieltur/my_awesome_catdog_model
ChasingMercer/beit-base
efederici/convnext-base-224-22k-1k-orig-cats-vs-dogs
Camilosan/Modelo-catsVSdogs
ismgar01/vit-base-cats-vs-dogs
nateraw/vit-base-cats-vs-dogs
cppgohan/resnet-50-finetuned-dog-vs-cat
tangocrazyguy/resnet-50-finetuned-cats_vs_dogs
akahana/vit-base-cats-vs-dogs
Amadeus99/cat_vs_dog_classifier
mhdiqbalpradipta/cat_or_dogs
Natalia2314/vit-base-catsVSdogs-demo-v5
Dricz/cat-vs-dog-resnet-50


In [13]:
# Prepare classification model

def create_classifier(model_id, device_num=0):
    return transformers.pipeline(model=model_id, device=device_num)

# def predict_image(classifier, file_path):
#     try:
#         image = Image.open(file_path)
#         image = image.convert('RGB')  # Ensure image is in RGB format
        
#         result = classifier(image)
#         predicted_label = result[0]['label'].split()[-1]
#         return predicted_label.lower()
#     except Exception as e:
#         print(f"Error processing file {file_path}: {str(e)}")
#         return None

def predict_images_batch(classifier, file_paths, batch_size=128):
    predictions = []
    for i in range(0, len(file_paths), batch_size):
        batch = file_paths[i:i+batch_size]
        images = [Image.open(file) for file in batch]
        # images = [Image.open(file).convert('RGB') for file in batch]
        try:
            batch_preds = classifier(images)
            batch_labels = [pred[0]['label'].split()[-1].lower() for pred in batch_preds]
            predictions.extend(batch_labels)
        except Exception as e:
            print(f"Error processing batch starting at index {i}: {str(e)}")
            predictions.extend([None] * len(batch))
    return predictions

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
# acc = []
# pre = []
# recall = []
# f1 = []

for index, model_info in cat_dog_models.iterrows():
    model_id = model_info['model'].id
    print(f"Setting up classifier for model: {model_id}")
    classifier = create_classifier(model_id, device_num=0)
    
    df["model_id"] = model_info['model'].id
    df["predicted_label"] = predict_images_batch(classifier, df["file_path"].tolist())

    print(f"Results with model {model_id}:")

    label_map = {'cat': 0, 'dog': 1}
    df['true_labels'] = df['label'].map(label_map)
    df['pred_labels'] = df['predicted_label'].map(label_map)

    # df['pred_labels'] = df['pred_labels'].fillna(-1)  # Handle NaNs by setting them to -1
    # df['pred_labels'] = df['pred_labels'].astype(int)
    csv_name = model_id.split("/")[0] + "-" + model_id.split("/")[1]
    df.to_csv(f"./evaluation_results/{csv_name}_evaluation_results.csv")
    
    # results = clf_metrics.compute(
    #     references=df['true_labels'].tolist(),
    #     predictions=df['pred_labels'].tolist()
    # )

    # # try:
    # acc.append(results['accuracy'])
    # pre.append(results['precision'])
    # recall.append(results['recall'])
    # f1.append(results['f1'])
    # exce

    # print(results["accuracy"])




Setting up classifier for model: danieltur/my_awesome_catdog_model


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Results with model danieltur/my_awesome_catdog_model:
Setting up classifier for model: ChasingMercer/beit-base


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(


Results with model ChasingMercer/beit-base:
Setting up classifier for model: efederici/convnext-base-224-22k-1k-orig-cats-vs-dogs


...

Results with model mhdiqbalpradipta/cat_or_dogs:
Setting up classifier for model: Natalia2314/vit-base-catsVSdogs-demo-v5


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Results with model Natalia2314/vit-base-catsVSdogs-demo-v5:
Setting up classifier for model: Dricz/cat-vs-dog-resnet-50


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Results with model Dricz/cat-vs-dog-resnet-50:


In [23]:
label_map = {'cat': 0, 'dog': 1}
df['true_labels'] = df['label'].map(label_map)
df['pred_labels'] = df['predicted_label'].map(label_map)

# df['pred_labels'] = df['pred_labels'].fillna(-1)  # Handle NaNs by setting them to -1
# df['pred_labels'] = df['pred_labels'].astype(int)
df.to_csv(f"{model_id}evaluation_results.csv")

In [40]:
import pandas as pd 

def evaluation_results(folder_path, metrics):
    results = {}
    for file in os.listdir(folder_path):
        try:
            file_path = os.path.join(folder_path, file)
            print(f"Current file: {file_path}")
            df = pd.read_csv(file_path)
            result = metrics.compute(
                references=df['true_labels'].tolist(),
                predictions=df['pred_labels'].tolist()
            )
            results[df["model_id"][0]] = result
            
        except IOError:
            print(f"Could not read file: {file}")
    return results



clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
results = evaluation_results("/workspaces/ModelHubTest/experiments/prediction_model/evaluation_results", clf_metrics)
# names = set(names)

results

Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/Camilosan-Modelo-catsVSdogs_evaluation_results.csv
Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/tangocrazyguy-resnet-50-finetuned-cats_vs_dogs_evaluation_results.csv
Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/Natalia2314-vit-base-catsVSdogs-demo-v5_evaluation_results.csv
Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/Dricz-cat-vs-dog-resnet-50_evaluation_results.csv
Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/akahana-vit-base-cats-vs-dogs_evaluation_results.csv
Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/nateraw-vit-base-cats-vs-dogs_evaluation_results.csv
Current file: /workspaces/ModelHubTest/experiments/prediction_model/evaluation_results/cppgohan-resnet-50-finetuned-dog-vs-cat_evaluation_

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Camilosan/Modelo-catsVSdogs': {'accuracy': 0.7128247650635711,
  'f1': 0.3558586484810911,
  'precision': 0.21725965177895534,
  'recall': 0.9828767123287672},
 'tangocrazyguy/resnet-50-finetuned-cats_vs_dogs': {'accuracy': 0.9187396351575456,
  'f1': 0.4965753424657534,
  'precision': 0.4965753424657534,
  'recall': 0.4965753424657534},
 'Natalia2314/vit-base-catsVSdogs-demo-v5': {'accuracy': 0.8606965174129353,
  'f1': 0.5135135135135135,
  'precision': 0.3575268817204301,
  'recall': 0.910958904109589},
 'Dricz/cat-vs-dog-resnet-50': {'accuracy': 0.9466556108347153,
  'f1': 0.5758241758241758,
  'precision': 0.803680981595092,
  'recall': 0.4486301369863014},
 'akahana/vit-base-cats-vs-dogs': {'accuracy': 0.7487562189054726,
  'f1': 0.3656664340544313,
  'precision': 0.22962313759859773,
  'recall': 0.8972602739726028},
 'nateraw/vit-base-cats-vs-dogs': {'accuracy': 0.685737976782753,
  'f1': 0.32119402985074624,
  'precision': 0.19450469992769343,
  'recall': 0.9212328767123288},

In [46]:
# cat_dog_models["pipeline_accuracy"] = cat_dog_models["model"].apply(
#     lambda id: results[id]["accuracy"] if id in results else -1
# )

# cat_dog_models['pipeline_recall'] = cat_dog_models['model'].apply(
#     lambda id: results[id]['recall'] if id in results else -1
# )

# cat_dog_models['pipeline_f1'] = cat_dog_models['model'].apply(
#     lambda id: results[id]['f1'] if id in results else -1
# )

cat_dog_models['pipeline_precision'] = cat_dog_models['model'].apply(
    lambda id: results[id]['precision'] if id in results else -1
)

cat_dog_models.to_csv("cat_dog_dataset.csv")


In [ ]:
# df['Status'] = df['Age'].apply(lambda x: 'Adult' if x >= 18 else 'Minor')

# for

df[]

model_ids = set(df_names['Name'])

# Add a new column 'Matched' to df_main that checks if each name is in df_names
df_main['Matched'] = df_main['Name'].apply(lambda name: 'Match' if name in names_set else 'No Match')

print(df_main)